In [1]:
from math import ceil, floor, sqrt
import numpy as np

In [2]:
import logging
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(message)s') # Additional args: %(asctime)s %(levelname)s
log.setLevel(logging.DEBUG) # Comment or modify to INFO to avoid verbose logs

In [3]:
# Storage mass constant
C = 10**12
# Number of dworks per 1 KAS (dworks are also known as sompis, and are the smallest KAS unit)
K = 10**8
# Maximal (standard) mass per Tx
M = 10**5
# Current standard mass-to-fee ratio is 1, meaning that a Tx with M mass is expected to pay a fee of M dworks 
MASS_FEE_RATIO = 1
# Standard mass for a Tx with maximal mass 
F = MASS_FEE_RATIO * M
# The threshold for which below it we give up the change and turn it into fee
CHANGE_THRESHOLD = F
# The maximum storage mass allowed to use for a single payment (which could be composed of several chained txs)
MAX_PAYMENT_MASS = 20 * M

In [4]:
SPK_STD_LEN = 36 # version len + p2pk len
SIG_STD_LEN = 66 # schnorr sig is 64 bytes + 2 op data bytes
OUT_SIZE = 8 + 2 + 8 + SPK_STD_LEN
IN_SIZE = 32 + 4 + 8 + SIG_STD_LEN + 8
TX_FIELDS_SIZE = 2 + 8 + 8 + 8 + 20 + 8 + 32 + 8

def estimated_tx_size(inputs_num, outputs_num):
    return TX_FIELDS_SIZE + inputs_num * IN_SIZE + outputs_num * OUT_SIZE 

"""
Calculation of transaction compute mass. For simplicity, the calculation assumes
standard signatures and scripts. However other than that it is accurate.
"""
def compute_mass(inputs, outputs):
    inputs_num, outputs_num = len(inputs), len(outputs)
    return 1000 * inputs_num + 10 * SPK_STD_LEN * outputs_num + estimated_tx_size(inputs_num, outputs_num)

In [5]:
"""
Calculates the negative component of the storage mass formula. Note that there is no dependency on output
values but only on their count. The code is designed to maximize precision and to avoid intermediate overflows.
"""
def negative_mass(inputs, outputs_num):
    inputs_num = len(inputs)
    if outputs_num == 1 or inputs_num == 1 or (outputs_num == 2 and inputs_num == 2):
        return sum(C//v for v in inputs)
    return inputs_num*(C//(sum(inputs)//inputs_num)) 

"""
Calculates the storage mass for the provided input/output collections 
"""
def storage_mass(inputs, outputs):
    N = negative_mass(inputs, outputs_num=len(outputs))
    P = sum(C//o for o in outputs)
    return max(P-N, 0)

"""
Calculates the overall mass of this transaction
"""
def mass(inputs, outputs):
    return max(storage_mass(inputs, outputs), compute_mass(inputs, outputs)) 

In [6]:
"""
Implements the algorithm for creating a minimal chain of transactions which can make 
the desired payment without passing storage mass limits locally per transaction
"""
def create_payment_chain(inputs, payment):
    txs = []
    while storage_mass(inputs, outputs=[payment, sum(inputs) - payment - F]) > M:
        T = sum(inputs) - F
        N = negative_mass(inputs, 2) 
        D = (M + N)*T/C
        if D**2 - 4*D < 0:
            # Indiactes that a single small input was used. Adding another input might help 
            raise Exception('The input is too small, try adding one more input (negative sqrt)')
        # Single step optimization, taking the smaller solution
        alpha = (D - sqrt(D**2 - 4*D))/(2*D) 
        # Round *up* in order to not increase the mass above M
        outputs = [ceil(alpha * T), T - ceil(alpha * T)]
        if inputs == outputs:
            # The process reached a point where the ceil round up prevents further progress.
            raise Exception('The inputs and the payment need to be closer in value (integer precision)')
        txs.append((inputs, outputs))
        log.debug(storage_mass(inputs, outputs))
        inputs = outputs
    txs.append((inputs, [payment, sum(inputs) - payment - F]))
    log.debug(storage_mass(*txs[-1]))
    return txs

### Optimal fee

Given a set of inputs and a payment amount, calculating the optimal fee requires solving the following optimization problem. The reader can skip the formal explanation and read the code directly.  

#### Notation

Let $P$ be the payment value and denote $I$ to be the set of input values. Let $S = \sum_{v \in I}v$. Denote $N_1, N_2$ to be the negative component of the storage mass assuming $1, 2$ outputs respectively (note that if $|I| > 2$ they can be different). Additionally, denote $U_1, U_2$ to be the compute mass assuming $1, 2$ outputs respectively. Finally, let $m$ be the unknown mass and let $\text{fee} = mR$ where $R$ is the mass-to-fee ratio. 

#### Optimization

There are two possible paths. The first is to have no change output which means using all remaining funds as fee. In this case we have 
$$m = \max \Big\lbrace \frac{C}{P} - N_1, U_1 \Big\rbrace, \,\, \text{fee} = S - P\text{.}$$
If $\text{fee} \ge mR$, this path can be considered. 

The second path is to have a change output. In this case we have
$$m=\frac{C}{P} + \frac{C}{S-P-mR} - N_2\text{;}$$
along with two additional constrains: (i) $m \ge U_2$; and (ii) $S-P > mR$. The above equation translates to $\big(m+N_2-C/P\big)\big(S-P-mR\big)-C=0$ which solutions for are
$$m=\frac{\Big(\frac{CR}{P}-N_2 R-P+S\Big) \pm \sqrt{\big(\frac{CR}{P}-N_2 R-P+S\big)^2+4R\big(N_2 S - N_2 P - \frac{CS}{P}\big)}}{2R}
\text{.}$$

The following algorithm implements this search: 

In [7]:
def optimal_fee(inputs, payment):
    P = payment
    S = sum(inputs)
    R = MASS_FEE_RATIO
    N1 = negative_mass(inputs, outputs_num=1)
    N2 = negative_mass(inputs, outputs_num=1)
    U1 = compute_mass(inputs, outputs=[payment])
    U2 = compute_mass(inputs, outputs=[payment - 1, 1])
    
    sol = []
    a = R
    b = (C*R)/P-N2*R-P+S  # Actually minus b
    c = N2*S-N2*P-(C*S)/P # Actually minus c
    n = b**2+4*a*c
    if n >= 0:
        sq = sqrt(n)
        m1 = max(floor((b-sq)/(2*a)), U2)
        m2 = max(floor((b+sq)/(2*a)), U2)
        if S-P > m1*R:
            sol.append((m1, U2, m1*R, S-P-m1*R)) # (mass, cmass, fee, change)
        if S-P > m2*R:
            sol.append((m2, U2, m2*R, S-P-m2*R)) # (mass, cmass, fee, change)
    m3 = max(C//P-N1, U1) # No change path
    if S-P >= m3*R:
        sol.append((m3, U1, S-P, 0)) # (mass, cmass, fee, change)
    log.debug(f'[optimal fee] inputs: %s, payment: %d, sol: %s', inputs, payment, sol)
    # By order of insertion, the first entry will always be the one with minimum fee 
    return sol[0] if sol else (None, None, None, None)

In [8]:
"""
Helper class for managing the available UTXO entries
"""
class Context:
    def __init__(self, utxos):
        # The full list of currently available UTXO entries
        self.utxos = utxos.copy()
        log.debug(f'[context::ctor] utxos: %s', self.utxos)
    
    def has_any(self):
        return len(self.utxos) > 0
    
    def has_above(self, threshold):
        utxos = np.array(self.utxos, dtype=np.uint64)
        return any(utxos >= threshold)
    
    # Returns the min UTXO above `threshold` or the max below it if no such value exists 
    def pop_min_above(self, threshold):
        utxos = np.array(self.utxos, dtype=np.uint64)
        large = utxos[utxos >= threshold]
        if len(large) > 0:
            item = min(large)
        else:
            item = max(utxos)
        self.utxos.remove(item)
        return item
    
    # Returns the max UTXO below `threshold` or the min above it if no such value exists
    def pop_max_below(self, threshold):
        utxos = np.array(self.utxos, dtype=np.uint64)
        small = utxos[utxos <= threshold]
        if len(small) > 0:
            item = max(small)
        else:
            item = min(utxos)
        self.utxos.remove(item)
        return item
    

In [9]:

"""
Main entry point for a send operation. Using the available `utxos`, creates transaction(s)
for sending the desired `amount`, while minimizing mass and fees as much as possible
"""
def create_transactions(utxos, payment):
    # Processing context
    ctx = Context(utxos)
    
    if ctx.has_above(threshold=payment):
        # Storage mass zone
        inputs = [ctx.pop_min_above(threshold=payment)]
        mass, cmass, fee, change = optimal_fee(inputs, payment)
        while ctx.has_any():
            if mass: # Indicates a solution exists
                if change > 0: # Indicates that fee is tight (with relation to mass)
                # if mass == cmass: # Indicates that adding inputs will not reduce the mass
                   break
            inputs.append(ctx.pop_max_below(payment))
            mass, cmass, fee, change = optimal_fee(inputs, payment)
    
        if mass is None:
            raise Exception('insufficient funds')
        if change == 0:
            raise Exception('make sure fee %d is ok with user'.format(fee))
            
        return create_payment_chain(inputs, payment)
        
            
    # Else -- compute mass zone -- all utxos are smaller than `payment`
    # In this case all current wallet algorithms should work without
    # modification, including compounding logic etc. The only exception
    # is the case where the change is randomly small which should be
    # handled either by adding inputs or by giving up the change and adding 
    # it as fee
    raise Exception('not implemented - storage mass logic is insignificant for compounding cases')

create_transactions([10*K, 2*K, 12*K, 3*K], int(K*0.1))

[context::ctor] utxos: [1000000000, 200000000, 1200000000, 300000000]
[optimal fee] inputs: [200000000], payment: 10000000, sol: [(100265, 2040, 100265, 189899735.0), (189994734, 2040, 189994734, 5266.0), (95000.0, 1626, 190000000.0, 0)]
99999.0
269.0


[([200000000], [10026739, 189873261.0]),
 ([10026739, 189873261.0], [10000000, 189800000.0])]

In [10]:
MASS_FEE_RATIO = 1
inputs = [K]*2 # [int(K*0.2)]*4
payment = int(K*0.5)
m, cm, fee, change = optimal_fee(inputs, payment)
m, mass(inputs, [payment, change]), mass(inputs, [payment, change - 1000000]), mass(inputs, [payment, change + 1000000])

[optimal fee] inputs: [100000000, 100000000], payment: 50000000, sol: [(6666, 3158, 6666, 149993334), (149993333, 3158, 149993333, 6667), (2744, 2744, 150000000, 0)]


(6666, 6666, 6711, 6622)

In [11]:
mass([int(K*0.02), int(K*0.02)], [int(K*0.01), 2719034])

367777

In [12]:
optimal_fee([K*10], K-int(K*0.01))

[optimal fee] inputs: [1000000000], payment: 99000000, sol: [(10210, 2040, 10210, 900989790), (900998890, 2040, 900998890, 1110), (9101, 1626, 901000000, 0)]


(10210, 2040, 10210, 900989790)